In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
from datetime import timedelta
import os
os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [2]:
keys = ['8to9', '9to10', '10to11', '11to12','12to13','13to14','14to15','15to16','16to17','17to18','18to19','19to20']


In [3]:
# read data
eod_original_cloud_ids = [201, 203, 206, 221, 229] 
eod_ema = pd.read_csv(dir + 'eod-ema.csv')
eod_ema = eod_ema.drop(['offset'], axis = 1)
eod_ema_alternative = pd.read_csv(dir + 'eod-ema-alternative.csv')
eod_ema_backup = pd.read_csv(dir + 'eod-ema-backup.csv')
temp_eod_original = eod_ema[eod_ema['participant_id'].isin(eod_original_cloud_ids)]
temp_eod_alt = eod_ema_alternative[~eod_ema_alternative['participant_id'].isin(eod_original_cloud_ids)]

eod_complete = pd.concat([temp_eod_original, temp_eod_alt, eod_ema_backup])

puffmarker_original_cloud_ids = [201, 203, 206, 210, 221, 229] 
puffmarker = pd.read_csv(dir + 'puff-probability.csv')
puffmarker = puffmarker.drop(['offset'], axis = 1)
puffmarker_alternative = pd.read_csv('puff-probability-alternative.csv')
puffmarker_backup = pd.read_csv('puff-probability-backup.csv')

temp_puffmarker_original = puffmarker[puffmarker['participant_id'].isin(puffmarker_original_cloud_ids)]
temp_puffmarker_alt = puffmarker_alternative[~puffmarker_alternative['participant_id'].isin(puffmarker_original_cloud_ids)]

puffmarker_complete = pd.concat([temp_puffmarker_original, temp_puffmarker_alt, puffmarker_backup])

participant_dates = pd.read_csv(dir + 'participant-dates.csv')

In [4]:
# Construct a list of id + dates for eod
# Use to look 
eod_dates = []
for irow in range(0,eod_complete.shape[0]):
    row = eod_complete.iloc[irow]
    if row['participant_id'] == 252 or row['participant_id'] == 253:
        continue
    quit_iloc = np.where(participant_dates['participant'] == row['participant_id'])
    quit_row = participant_dates.iloc[quit_iloc[0][0]]
    quit_time = datetime.datetime.strptime(quit_row['quit_date'], '%m/%d/%y')
    if row['status'] == "MISSED":
        continue
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    #print row['participant_id'], time
    #print quit_row['participant'], quit_time
    temp_diff = time - quit_time
    if time.hour  == 0 or time.hour == 1:
        date = np.array([row['participant_id'], temp_diff.days-1])
        date = np.append(date, np.sum(np.array(row[keys])))
    else:
        date = np.array([row['participant_id'], temp_diff.days])
        date = np.append(date, np.sum(np.array(row[keys])))
    #print date
    eod_dates.append(date)
    
eod_dates = np.asarray(eod_dates)

In [5]:
eod_survival = []
for id in set(eod_dates[:,0]):
    subset = eod_dates[eod_dates[:,0] == id,:]
    if any(subset[:,1] > 0):
        subset = subset[subset[:,1] > 0,:]
        nonzero_iloc = np.where(subset[:,2] > 0)[0]
        zero_iloc = np.where(subset[:,2] == 0)[0]
        if len(nonzero_iloc) == 0:
            if len(zero_iloc) != 0:
                temp = np.append(subset[np.max(zero_iloc),0:2], 0)
        else:
            temp = np.append(subset[np.max(nonzero_iloc), 0:2], 1)
        eod_survival.append(temp)

eod_survival = np.asarray(eod_survival)

In [6]:
# Read data for contingent EMA
ec_original_cloud_ids = [201, 221, 229] 
contingent_ema = pd.read_csv(dir + 'eventcontingent-ema.csv')
contingent_ema = contingent_ema.drop(['offset'], axis = 1)
contingent_ema_alternative = pd.read_csv(dir + 'eventcontingent-ema-alternative.csv')
contingent_ema_backup = pd.read_csv(dir + 'eventcontingent-ema-backup.csv')

temp_contingent_original = contingent_ema[contingent_ema['participant_id'].isin(ec_original_cloud_ids)]
temp_contingent_alt = contingent_ema_alternative[~contingent_ema_alternative['participant_id'].isin(ec_original_cloud_ids)]

contingent_complete = pd.concat([temp_contingent_original, temp_contingent_alt, contingent_ema_backup])

In [7]:
# Use to look 
ec_dates = []
for irow in range(0,contingent_complete.shape[0]):
    row = contingent_complete.iloc[irow]
    quit_iloc = np.where(participant_dates['participant'] == row['participant_id'])
    quit_row = participant_dates.iloc[quit_iloc[0][0]]
    quit_time = datetime.datetime.strptime(quit_row['quit_date'], '%m/%d/%y')
    if row['status'] == "MISSED":
        continue
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    #print row['participant_id'], time
    #print quit_row['participant'], quit_time
    temp_diff = time - quit_time
    if row['smoke'] == 'Yes':
        if time.hour  == 0 or time.hour == 1:
            date = np.array([row['participant_id'], temp_diff.days-1])
        else:
            date = np.array([row['participant_id'], temp_diff.days])
    #print date, time
        ec_dates.append(date)
    
ec_dates = np.asarray(ec_dates)

In [8]:
ec_survival = []
for id in set(ec_dates[:,0]):
    subset = ec_dates[ec_dates[:,0] == id,:]
    if any(subset[:,1] > 0):
        subset = subset[subset[:,1] > 0,:]
        temp = [id, np.max(subset[:,1])]
        ec_survival.append(temp)

ec_survival = np.asarray(ec_survival)

In [9]:
# Read data for random EMA
random_original_cloud_ids = [201, 203, 206, 210, 221, 226, 229] 
random_ema = pd.read_csv(dir + 'random-ema.csv')
random_ema = random_ema.drop(['offset'], axis = 1)

random_ema_alternative = pd.read_csv(dir + 'random-ema-alternative.csv')
random_ema_backup = pd.read_csv(dir + 'random-ema-backup.csv')

temp_random_original = random_ema[random_ema['participant_id'].isin(random_original_cloud_ids)]
temp_random_alt = random_ema_alternative[~random_ema_alternative['participant_id'].isin(random_original_cloud_ids)]

random_complete = pd.concat([temp_random_original, temp_random_alt, random_ema_backup])

In [10]:
# Use to look 
random_dates = []
for irow in range(0,random_complete.shape[0]):
    row = random_complete.iloc[irow]
    quit_iloc = np.where(participant_dates['participant'] == row['participant_id'])
    quit_row = participant_dates.iloc[quit_iloc[0][0]]
    quit_time = datetime.datetime.strptime(quit_row['quit_date'], '%m/%d/%y')
    if row['status'] == "MISSED":
        continue
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    #print row['participant_id'], time
    #print quit_row['participant'], quit_time
    temp_diff = time - quit_time
    if time.hour  == 0 or time.hour == 1:
        date = np.array([row['participant_id'], temp_diff.days-1])
        if row['smoke'] == 'Yes':
            date = np.append(date, [1])
            random_dates.append(date)
        if row['smoke'] == 'No':
            date = np.append(date, [0])
            random_dates.append(date)
    else:
        date = np.array([row['participant_id'], temp_diff.days])
        if row['smoke'] == 'Yes':
            date = np.append(date, 1)
            random_dates.append(date)
        if row['smoke'] == 'No':
            date = np.append(date, 0)
            random_dates.append(date)

random_dates = np.asarray(random_dates)

In [11]:
random_survival = []
for id in set(random_dates[:,0]):
    subset = random_dates[random_dates[:,0] == id,:]
    if any(subset[:,1] > 0):
        subset = subset[subset[:,1] > 0,:]
        nonzero_iloc = np.where(subset[:,2] == 1)[0]
        zero_iloc = np.where(subset[:,2] == 0)[0]
        if len(nonzero_iloc) == 0:
            if len(zero_iloc) != 0:
                temp = np.append(subset[np.max(zero_iloc),0:2], 0)
        else:
            temp = np.append(subset[np.max(nonzero_iloc), 0:2], 1)
        random_survival.append(temp)

random_survival = np.asarray(random_survival)

In [12]:
all_ema_survival = []

for id in set(random_survival[:,0]) | set(ec_survival[:,0]) | set(eod_survival[:,0]):
    temp = np.array([id])
    random_subset = random_survival[random_survival[:,0] == id,:]
    if not len(random_subset) == 0:
        temp = np.append(temp, random_subset[0][1:3])
    else:
        temp = np.append(temp, [-1,-1])
    ec_subset = ec_survival[ec_survival[:,0] == id,:]
    if not len(ec_subset) == 0:
        ec_day = ec_subset[0][1]
        if temp[1] > ec_day or temp[1] == -1:
            temp[1:3] = [ec_day,1]
    eod_subset = eod_survival[eod_survival[:,0] == id, :]
    if not len(eod_subset) == 0:
        if eod_subset[0][1] < temp[1] and eod_subset[0][2] == 1:
            temp[1:3] = eod_subset[0][1:3]
        elif eod_subset[0][1] > temp[1] and eod_subset[0][2] == 0 and temp[2] == 0:
            temp[1:3] = eod_subset[0][1:3]
    all_ema_survival.append(temp)
    
all_ema_survival = np.asarray(all_ema_survival)

all_ema_survival

array([[202,   7,   1],
       [203,  13,   0],
       [205,   8,   1],
       [206,   9,   0],
       [207,   1,   1],
       [208,   3,   1],
       [211,   2,   1],
       [212,   6,   1],
       [213,   1,   0],
       [214,   2,   1],
       [215,  10,   1],
       [216,   9,   1],
       [217,   3,   1],
       [218,   1,   1],
       [219,  10,   1],
       [222,   1,   1],
       [223,  11,   0],
       [224,   9,   1],
       [225,   9,   1],
       [226,   5,   1],
       [227,  10,   1],
       [228,   8,   1],
       [229,   2,   1],
       [230,   9,   1],
       [231,   1,   1],
       [233,   6,   1],
       [234,   1,   0],
       [235,   4,   0],
       [238,   9,   1],
       [240,  10,   1],
       [241,   1,   1],
       [242,   8,   1],
       [243,   5,   0],
       [245,   4,   1],
       [247,   9,   1],
       [248,  10,   0],
       [249,   1,   1],
       [250,  10,   0],
       [252,   9,   1],
       [253,  11,   1]])

In [13]:
puffmarker_episode_original_cloud_ids = [201, 203, 206, 210, 229] 
puffmarker_episode = pd.read_csv(dir + 'puff-episode.csv')
puffmarker_episode = puffmarker_episode.drop(['offset'], axis = 1)
puffmarker_episode_alternative = pd.read_csv('puff-episode-alternative.csv')
puffmarker_episode_backup = pd.read_csv('puff-episode-backup.csv')

temp_puffmarker_episode_original = puffmarker_episode[puffmarker_episode['participant_id'].isin(puffmarker_episode_original_cloud_ids)]
temp_puffmarker_episode_alt = puffmarker_episode_alternative[~puffmarker_episode_alternative['participant_id'].isin(puffmarker_episode_original_cloud_ids)]

puffmarker_episode_complete = pd.concat([temp_puffmarker_episode_original, temp_puffmarker_episode_alt, puffmarker_episode_backup])


In [14]:
## We now have RANDOM, EC, EOD, puffMarker
## Still need to bring in the HTMGs.
participant_id = 252
delta = 9
quit_iloc = np.where(participant_dates['participant'] == participant_id)
quit_row = participant_dates.iloc[quit_iloc[0][0]]
quit_time = datetime.datetime.strptime(quit_row['quit_date'], '%m/%d/%y')
current_date = quit_time + timedelta(days=delta)
print "Quit date is %s" % quit_time
print "Current date is %s" % current_date

eod_iloc = np.where(eod_complete['participant_id'] == participant_id)
eod_rows = eod_complete.iloc[eod_iloc[0]]
eod_rows = pd.DataFrame.from_dict(eod_rows)

for irow in range(0,eod_rows.shape[0]):
    row = eod_rows.iloc[irow]
    date = row['date']
    try:
        time = datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    temp = time - current_date
    if time.hour  == 0 or time.hour == 1:
        continue
    if temp.days == 0:
        print eod_rows.iloc[irow]

random_iloc = np.where(random_complete['participant_id'] == participant_id)
random_rows = random_complete.iloc[random_iloc[0]]
random_rows = pd.DataFrame(random_rows)
print 'Onto RANDOM EMA'

for irow in range(0,random_rows.shape[0]):
    row = random_rows.iloc[irow]
    date = row['date']
    try:
        time = datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    temp = time - current_date
    if temp.days == 0:
        print row['smoke'], row['when_smoke'], row['date']

contingent_iloc = np.where(contingent_complete['participant_id'] == participant_id)
contingent_rows = contingent_complete.iloc[contingent_iloc[0]]
contingent_rows = pd.DataFrame(contingent_rows)
print 'Onto Event Contingent EMA'

for irow in range(0,contingent_rows.shape[0]):
    row = contingent_rows.iloc[irow]
    date = row['date']
    try:
        time = datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    temp = time - current_date
    if temp.days == 0:
        print row['smoke'], row['when_smoke'], row['date']

puffmarker_episode_iloc = np.where(puffmarker_episode_complete['participant_id'] == participant_id)
puffmarker_episode_rows = puffmarker_episode_complete.iloc[puffmarker_episode_iloc[0]]
puffmarker_episode_rows = pd.DataFrame(puffmarker_episode_rows)
print 'Onto puffMarker episode detection'

for irow in range(0,puffmarker_episode_rows.shape[0]):
    row = puffmarker_episode_rows.iloc[irow]
    date = row['date']
    try:
        time = datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    temp = time - current_date
    if temp.days == 0:
        print row['date']

puffmarker_iloc = np.where(puffmarker_complete['participant_id'] == participant_id)
puffmarker_rows = puffmarker_complete.iloc[puffmarker_iloc[0]]
puffmarker_rows = pd.DataFrame(puffmarker_rows)
print 'Onto puffMarker puff detection'

total = 0
for irow in range(0,puffmarker_rows.shape[0]):
    row = puffmarker_rows.iloc[irow]
    date = row['date']
    try:
        time = datetime.datetime.strptime(date, '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    temp = time - current_date
    if temp.days == 0:
        total += 1
        print row['date']

print "Total HTMGs = %s" %total

Quit date is 2018-12-03 00:00:00
Current date is 2018-12-12 00:00:00
status                      COMPLETED
8to9                                1
9to10                               0
10to11                              0
11to12                              1
12to13                              1
13to14                              0
14to15                              1
15to16                              0
16to17                              0
17to18                              0
18to19                              0
19to20                              0
participant_id                    252
timestamp               1544668414661
date              2018-12-12 20:33:34
hour                               20
minute                             33
day_of_week                 Wednesday
Name: 320, dtype: object
Onto RANDOM EMA
Yes 1 - 19 Minutes 2018-12-12 07:16:10
No None 2018-12-12 12:05:20
No None 2018-12-12 15:48:05
Onto Event Contingent EMA
Yes 15 to 30 minutes 2018-12-12 11:35:53
Onto p

In [43]:
from math import *
smoking_rate = 1./4.5
print 4*np.log(smoking_rate)-smoking_rate
print 5*np.log(smoking_rate)-smoking_rate


num_htmgs = 72
lambda_parameter = 1/(3.5*6.)
lambda_prime_hour = 1/(1.25*6.)
window_lambda = lambda_parameter+lambda_prime_hour
seq1 = [4., 4., 8.,3.,3.]
remaining_htmgs1 = num_htmgs - np.sum(seq1)
time_in_smoking1 = 1./6. * len(seq1)
remaining_time1 = (20.5-7) - 1./6. * len(seq1)

total1 = remaining_htmgs1*np.log(lambda_prime_hour) - lambda_prime_hour*remaining_time1
total1 += np.sum(seq1)*np.log(window_lambda) - window_lambda*time_in_smoking1
print total1

seq2 = [4., 8.,3.,3.]
remaining_htmgs2 = num_htmgs - np.sum(seq2)
time_in_smoking2 = 1./6. * len(seq2)
remaining_time2 = (20.5-7) - 1./6. * len(seq2)

total2 = remaining_htmgs2*np.log(lambda_prime_hour) - lambda_prime_hour*remaining_time2
total2 += np.sum(seq2)*np.log(window_lambda) - window_lambda*time_in_smoking2
print total2

temp = -5.29677-141.40789-3.6177 - (-6.54952-140.19430-3.0530)
print temp
print 1 - np.exp(temp)
print temp - log(1-0.41)


-6.238531809327319
-7.742609206103594
-140.1943037285996
-141.40789381886782
-0.52554
0.4087639892967142
0.00209274208234
